In [2]:
# -*- coding:utf-8 -*-
import webbrowser
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

# gen url to jisho
def webInfo(kan):
    webstem = 'https://jisho.org/search/'
    suffix = ""
    k = str((kan).encode("utf-8"))
    kanji = k[2:-1]
    chars = "\\x"
    kanji = kanji.replace(chars, '%')
    webstring = webstem + kanji + suffix
    return webstring


# Load webpage content
def jlptGrade(webstring):
    r = requests.get(webstring)
    soup = bs(r.content, features="html.parser")
    try:
        n = soup.find("span", string=re.compile('(JLPT|jlpt)')).text
        grade = n.split(' ')[1]
    except:
        grade = 'NA'
    return grade

In [20]:
# get csv to df for preparation
path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/word_dictionary.csv'
df = pd.read_csv(path)
kanjis = df['kanji']




In [ ]:
# scrape grades
grades = []
for i,kanji in enumerate(kanjis):
    if i % 50 ==0:
        print(i, kanji)
    webstring = webInfo(kanji)
    grade = jlptGrade(webstring)
    grades.append(grade)

print(grades)


In [23]:
df['grade'] = grades
df.grade.value_counts()

N5    234
N4    117
NA     83
N3     80
N2     26
N1     23
Name: grade, dtype: int64

In [27]:
# get streak info
path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/streak.csv'
df_str = pd.read_csv(path)
df['streak'] = df_str.streak
df['level'] = df.streak.apply(lambda x: 'expert' if x > 7 else 'advanced' if x > 2 else 'beginner')

In [82]:
# get results info
path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/results2.csv'
df_res = pd.read_csv(path)
fails = df_res.groupby('index_word')['result'].value_counts().unstack().fillna(0)[0]
successes = df_res.groupby('index_word')['result'].value_counts().unstack().fillna(0)[1]
df['total_attempts'] = (fails + successes)
df['total_successes'] = successes
df['total_fails'] = fails
df

     index_word  attempts
398           0        19
524           1        12
444           2        17
15            3        49
30            4        42
220           5        26
198           6        27
18            7        47
107           8        33
91            9        34
208          10        26
436          11        17
161          12        29
348          13        21
103          14        33
435          15        17
476          16        15
92           17        34
467          18        16
271          19        24


,word index,english,hiragana,kanji,date_added,rank,sample,is_active,grade,streak,level,total_attempts,total_successes,total_fails
0,0,house,いえ,家,09/05/2020,1,1,0,N5,10,expert,19.0,19.0,0.0
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0,N4,10,expert,12.0,12.0,0.0
2,2,nervous,ふあん,不安,09/05/2020,2,1,0,N3,10,expert,17.0,17.0,0.0
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1,N5,8,expert,49.0,40.0,9.0
4,4,to lend,かす,貸す,09/05/2020,2,1,0,N5,10,expert,42.0,35.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,558,entirely,まったく,全く,22/12/2020,1,1,1,N3,3,advanced,5.0,5.0,0.0
559,559,(something) drops,おちる,落ちる,22/12/2020,3,1,1,N4,2,beginner,7.0,5.0,2.0
560,560,to tidy up,かたづける,片付ける,22/12/2020,4,1,1,N4,0,beginner,4.0,0.0,4.0
561,561,(something) goes off,きえる,消える,22/12/2020,3,1,1,N5,0,beginner,9.0,3.0,6.0


In [15]:
# age column
from datetime import datetime as dt
df = pd.read_csv('kanji_data_cleansed.csv')
today_ = pd.to_datetime('today').normalize()

df['age'] = df.date_added.apply(lambda x: today_ - pd.to_datetime(dt.strptime(x,"%d/%m/%Y"))).dt.days
df

,word index,english,hiragana,kanji,date_added,rank,sample,is_active,grade,streak,level,total_attempts,total_successes,total_fails,age
0,0,house,いえ,家,09/05/2020,1,1,0,N5,10,expert,19.0,19.0,0.0,234
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0,N4,10,expert,12.0,12.0,0.0,234
2,2,nervous,ふあん,不安,09/05/2020,2,1,0,N3,10,expert,17.0,17.0,0.0,234
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1,N5,8,expert,49.0,40.0,9.0,234
4,4,to lend,かす,貸す,09/05/2020,2,1,0,N5,10,expert,42.0,35.0,7.0,234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,558,entirely,まったく,全く,22/12/2020,1,1,1,N3,3,advanced,5.0,5.0,0.0,7
559,559,(something) drops,おちる,落ちる,22/12/2020,3,1,1,N4,2,beginner,7.0,5.0,2.0,7
560,560,to tidy up,かたづける,片付ける,22/12/2020,4,1,1,N4,0,beginner,4.0,0.0,4.0,7
561,561,(something) goes off,きえる,消える,22/12/2020,3,1,1,N5,0,beginner,9.0,3.0,6.0,7


In [16]:
df.to_csv('kanji_data_cleansed.csv', index=False)

In [17]:
df

,word index,english,hiragana,kanji,date_added,rank,sample,is_active,grade,streak,level,total_attempts,total_successes,total_fails,age
0,0,house,いえ,家,09/05/2020,1,1,0,N5,10,expert,19.0,19.0,0.0,234
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0,N4,10,expert,12.0,12.0,0.0,234
2,2,nervous,ふあん,不安,09/05/2020,2,1,0,N3,10,expert,17.0,17.0,0.0,234
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1,N5,8,expert,49.0,40.0,9.0,234
4,4,to lend,かす,貸す,09/05/2020,2,1,0,N5,10,expert,42.0,35.0,7.0,234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,558,entirely,まったく,全く,22/12/2020,1,1,1,N3,3,advanced,5.0,5.0,0.0,7
559,559,(something) drops,おちる,落ちる,22/12/2020,3,1,1,N4,2,beginner,7.0,5.0,2.0,7
560,560,to tidy up,かたづける,片付ける,22/12/2020,4,1,1,N4,0,beginner,4.0,0.0,4.0,7
561,561,(something) goes off,きえる,消える,22/12/2020,3,1,1,N5,0,beginner,9.0,3.0,6.0,7
